<a href="https://colab.research.google.com/github/camayli/Trabajo-Final-Aprendiza-Autom-tico-II-/blob/main/Modelo_N%C2%B02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Patricia

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

import kagglehub
from kagglehub import KaggleDatasetAdapter
from sklearn.model_selection import train_test_split

##**1. Carga de datos**

In [11]:
!pip install -q kagglehub[pandas-datasets]

warnings.filterwarnings("ignore", category=FutureWarning)

file_path = "water_potability.csv"

df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "adityakadiwal/water-potability",
    file_path,
)

/tmp/ipython-input-3234254166.py:7: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


Using Colab cache for faster access to the 'water-potability' dataset.


In [12]:
df.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


##**2. Análisis Exploratorio de Datos**

In [ ]:
print(df.shape)

In [ ]:
print(df.info())

In [13]:
print(df['Potability'].value_counts())

Potability
0    1998
1    1278
Name: count, dtype: int64


In [ ]:
df.describe()

In [ ]:
df_num = df.select_dtypes(include="number")
df_long = df_num.melt(var_name="Variable",value_name="Valor")

g = sns.FacetGrid(
    df_long,
    col="Variable",
    col_wrap=4,
    sharex=False,
    sharey=False
)

g.map_dataframe(
    sns.histplot,
    x="Valor",
    bins=30,color="#A4C2F4", edgecolor="#0B3C5D")

g.set_axis_labels("Valor", "Frecuencia")

g.fig.suptitle(
    "Histogramas de las variables del estudio",
    fontsize=16,
    fontweight="bold"
)

for ax in g.axes.flatten():
    ax.tick_params(labelsize=11)
    ax.set_title(ax.get_title(), fontsize=13)

plt.tight_layout()
plt.show()

##**3. Preprocesamiento de datos**

### **3.1. Balanceo de clases**

#### **3.1.1. Separación train/test -- Valores nulos -- Imputación**

> **Clases**

In [5]:
print(df['Potability'].value_counts())

Potability
0    1998
1    1278
Name: count, dtype: int64


> **Separar variables**

In [6]:
X = df.drop(columns='Potability')
y = df['Potability']

> **División Train /Test**

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

> **Imputamos valores nulos mediante KNN Imputation (k=7)**

In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

imputer = KNNImputer(
    n_neighbors=7,        # puedes probar 3, 5, 7
    weights="distance"    # recomendado
)

X_train = imputer.fit_transform(X_train)
X_test  = imputer.transform(X_test)

#### **3.1.2. Balanceo: SMOTE, ADASYN, OVERSAMPLING**

In [31]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, f1_score

from imblearn.over_sampling import SMOTE, ADASYN, RandomOverSampler

# ======================================================
# MODELOS
# ======================================================
models = {
    "LogisticR": LogisticRegression(max_iter=1000),
    "RF": RandomForestClassifier(random_state=42),
    "SVM": SVC()
}

# ======================================================
# FUNCIÓN DE EVALUACIÓN COMPLETA
# ======================================================
def evaluate_models_full(X_train_bal, y_train_bal, X_test, y_test):
    metrics = {}

    for name, model in models.items():
        model.fit(X_train_bal, y_train_bal)
        y_pred = model.predict(X_test)

        metrics[f"Accuracy_{name}"] = accuracy_score(y_test, y_pred)
        metrics[f"Recall_{name}"]   = recall_score(y_test, y_pred)
        metrics[f"F1_{name}"]       = f1_score(y_test, y_pred)

    return metrics

# ======================================================
# LISTA DE RESULTADOS
# ======================================================
results = []

# ======================================================
# SMOTE k=3
# ======================================================
smote_3 = SMOTE(k_neighbors=3, random_state=42)
X_s3, y_s3 = smote_3.fit_resample(X_train, y_train)

row = evaluate_models_full(X_s3, y_s3, X_test, y_test)
row["Balanceo"] = "SMOTE_k3"
results.append(row)

# ======================================================
# SMOTE k=7
# ======================================================
smote_7 = SMOTE(k_neighbors=7, random_state=42)
X_s7, y_s7 = smote_7.fit_resample(X_train, y_train)

row = evaluate_models_full(X_s7, y_s7, X_test, y_test)
row["Balanceo"] = "SMOTE_k7"
results.append(row)

# ======================================================
# ADASYN k=5
# ======================================================
adasyn_5 = ADASYN(n_neighbors=5, random_state=42)
X_a5, y_a5 = adasyn_5.fit_resample(X_train, y_train)

row = evaluate_models_full(X_a5, y_a5, X_test, y_test)
row["Balanceo"] = "ADASYN_k5"
results.append(row)

# ======================================================
# ADASYN k=9
# ======================================================
adasyn_9 = ADASYN(n_neighbors=9, random_state=42)
X_a9, y_a9 = adasyn_9.fit_resample(X_train, y_train)

row = evaluate_models_full(X_a9, y_a9, X_test, y_test)
row["Balanceo"] = "ADASYN_k9"
results.append(row)

# ======================================================
# RANDOM OVERSAMPLING
# ======================================================
ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_resample(X_train, y_train)

row = evaluate_models_full(X_ros, y_ros, X_test, y_test)
row["Balanceo"] = "RandomOver"
results.append(row)

# ======================================================
# TABLA FINAL
# ======================================================
results_df = pd.DataFrame(results)
results_df = results_df.set_index("Balanceo")

print("TABLA FINAL DE RESULTADOS:")
results_df

TABLA FINAL DE RESULTADOS:


,Accuracy_LogisticR,Recall_LogisticR,F1_LogisticR,Accuracy_RF,Recall_RF,F1_RF,Accuracy_SVM,Recall_SVM,F1_SVM
Balanceo,,,,,,,,,
SMOTE_k3,0.545732,0.585938,0.501672,0.660061,0.476562,0.522484,0.608232,0.519531,0.508604
SMOTE_k7,0.527439,0.558594,0.479866,0.646341,0.449219,0.497835,0.628049,0.535156,0.528958
ADASYN_k5,0.466463,0.664062,0.492754,0.640244,0.460938,0.500000,0.586890,0.613281,0.536752
ADASYN_k9,0.432927,0.820312,0.530303,0.647866,0.507812,0.529532,0.574695,0.660156,0.547812
RandomOver,0.518293,0.511719,0.453287,0.670732,0.390625,0.480769,0.617378,0.480469,0.494970


| Balanceo   | Modelo        | Accuracy | Recall | F1    |
| ---------- | ------------- | -------- | ------ | ----- |
| ADASYN_k9  | Random Forest | 0.648    | 0.508  | 0.530 |
| ADASYN_k9  | SVM           | 0.575    | 0.660  | 0.548 |
| RandomOver | Random Forest | 0.671    | 0.391  | 0.481 |

* Mejor equilibrio global: ADASYN_k9 + Random Forest
* Mejor detección de positivos (recall): ADASYN_k9 + SVM
* Mayor accuracy (engañoso): RandomOver + Random Forest